In [2]:
from sklearn import *
import pandas as pd


In [3]:
# Read textfiles into dataframes
df_smoker = pd.read_csv('./data/internal_smokerslung.expression.txt', sep="\t", index_col=0).transpose()
df_non_smoker = pd.read_csv('./data/internal_nonsmokerslung.expression.txt', sep="\t", index_col=0).transpose()
df_smoker = df_smoker.drop(df_smoker.columns[0], axis=1)
df_smoker = df_smoker.reset_index(drop=True);
df_non_smoker = df_non_smoker.drop(df_non_smoker.columns[0], axis=1)
df_non_smoker = df_non_smoker.reset_index(drop=True);
print ("DF SMOKER");
print(df_smoker);
print("DF NON-SMOKER");
print(df_non_smoker);

DF SMOKER
       TMPRSS2  IL6R  IL6ST  PCSK1   IL6  PCSK2  CTSE  MYRF  MAG  MOG  ...  \
0          0.0   0.0    0.0    0.0   0.0    0.0   0.0   0.0  0.0  0.0  ...   
1          0.0   0.0    4.0    0.0  14.0    0.0   0.0   0.0  0.0  0.0  ...   
2          0.0   0.0    0.0    0.0  20.0    0.0   0.0   0.0  0.0  0.0  ...   
3          0.0   1.0    0.0    0.0   1.0    0.0   0.0   0.0  0.0  0.0  ...   
4          0.0   0.0    0.0    0.0   0.0    0.0   0.0   0.0  0.0  0.0  ...   
...        ...   ...    ...    ...   ...    ...   ...   ...  ...  ...  ...   
96781      0.0   0.0    3.0    0.0   0.0    0.0   0.0   0.0  0.0  0.0  ...   
96782      1.0   0.0    3.0    0.0   0.0    0.0   0.0   0.0  0.0  0.0  ...   
96783      0.0   0.0    0.0    0.0   0.0    0.0   0.0   0.0  0.0  0.0  ...   
96784      0.0   0.0    0.0    0.0   0.0    0.0   0.0   0.0  0.0  0.0  ...   
96785      0.0   1.0    1.0    0.0   0.0    0.0   0.0   0.0  0.0  0.0  ...   

       PCSK7  C1R   C2   C3   C5  CFI  CTSS  CTSL  CT

In [4]:
# countnumber of smoker and non-smoker samples
n_smoker = len(df_smoker)
n_non_smoker = len(df_non_smoker)
print(n_smoker);
print(n_non_smoker);
# resample the non-smoker data to have the same number of samples as the smoker data
df_non_smoker_resampled = df_non_smoker.sample(n=n_smoker, replace=False, random_state=42);
# label the data as smoke=yes or smoke=no in both dataframes
df_smoker['smoker'] = 'yes';
df_non_smoker_resampled['smoker'] = 'no';
# combine the data frames
df_combo_balanced = pd.concat([df_smoker, df_non_smoker_resampled], axis=0, ignore_index=True);
X = df_combo_balanced.drop(['smoker'], axis=1)
y = df_combo_balanced['smoker']
print('BALANCED');
print(X);

96786
277224
BALANCED
        TMPRSS2  IL6R  IL6ST  PCSK1   IL6  PCSK2  CTSE  MYRF  MAG  MOG  ...  \
0           0.0   0.0    0.0    0.0   0.0    0.0   0.0   0.0  0.0  0.0  ...   
1           0.0   0.0    4.0    0.0  14.0    0.0   0.0   0.0  0.0  0.0  ...   
2           0.0   0.0    0.0    0.0  20.0    0.0   0.0   0.0  0.0  0.0  ...   
3           0.0   1.0    0.0    0.0   1.0    0.0   0.0   0.0  0.0  0.0  ...   
4           0.0   0.0    0.0    0.0   0.0    0.0   0.0   0.0  0.0  0.0  ...   
...         ...   ...    ...    ...   ...    ...   ...   ...  ...  ...  ...   
193567      0.0   0.0    0.0    0.0   0.0    0.0   0.0   0.0  0.0  0.0  ...   
193568      0.0   0.0    0.0    0.0   9.0    0.0   0.0   0.0  0.0  0.0  ...   
193569      1.0   0.0    0.0    0.0   0.0    0.0   0.0   0.0  0.0  0.0  ...   
193570      0.0   0.0    2.0    0.0   0.0    0.0   0.0   0.0  0.0  0.0  ...   
193571      0.0   0.0    1.0    0.0   0.0    0.0   0.0   0.0  0.0  0.0  ...   

        PCSK7   C1R   C2   C3

In [5]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
# Encode the target using LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

# Normalize features using StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [6]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# dummy classifier
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_train, y_train)
y_pred = dummy_clf.predict(X_test)

# accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.49870849979335996


In [ ]:
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.svm import SVC

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42);

svm_model = svm.SVC(kernel='linear', probability=False);

scores = cross_val_score(svm_model, X_train, y_train, cv=5);
print(scores);